# 10. 3D case

Until now we have exclusively processed 2D images, eventhough the sometimes came from 3D acquisition. We are now going to look at an example of 3D processing where we are going to use the same tools as in 2D but in a 3D version.

Extending an image processing pipeline from 2D to 3D can be challenging for two reasons: first, computations can become very slow because of the amount of data changes usually roughly by an order of magnitude, and second, visualization of both original and processed data is more complicated.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.external.tifffile import TiffFile
import skimage.io as io

## 10.1 Visualization

Developing tools to visualize 3D data is a non-trivial taks becasuse of the amount of data to handle and because of the difficulty to properly handle volume visualization using projections and transparency. We present here two possibilities that integrate well with the Jupyter environment: ipyvolume and itk

In [2]:
image = io.imread('/Users/gw18g940/Desktop/ImageProcessingLectureData/BBBC034_v1_dataset/AICS_12_134_C=2.tif')

### 10.1.1 ipyvolume

In [3]:
import ipyvolume.pylab as ipv

In [4]:
ipv.figure()
ipv.volshow(image, extent=[[0,image.shape[1]],[0,image.shape[2]],[0, 5*image.shape[0]]],
           level = [0,0,0.27], opacity=[0,0,0.1], max_opacity=1)
ipv.xlim(0,image.shape[1])
ipv.ylim(0,image.shape[1])
ipv.zlim(-image.shape[1]/2,image.shape[1]/2)
ipv.show()


/Users/gw18g940/miniconda3/envs/teaching_test/lib/python3.6/site-packages/ipyvolume/widgets.py:179: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_view = self.data_original[view]
/Users/gw18g940/miniconda3/envs/teaching_test/lib/python3.6/site-packages/ipyvolume/utils.py:204: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = (data[slices1] + data[slices2])/2
/Users/gw18g940/miniconda3/envs/teaching_test/lib/python3.6/site-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradi

### 10.1.2 itk-widget

In [3]:
from itkwidgets import view

In [4]:
image.shape

(52, 1024, 1024)

In [5]:
from skimage.transform import rescale

In [9]:
image2.shape

(260, 1024, 1024)

In [16]:
image2 = rescale(image,scale=(1,0.2,0.2),multichannel=False,order=1,preserve_range=True)

The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.


In [18]:
type(image2[0,0,0])

numpy.float64

In [20]:
view(image2.astype(np.uint16))

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageUS3; proxy of <Swig Object of type 'itkIm…